# Reads ERA5-Land and Copernicus Global Land Cover data already available as a netCDF file stored locally
# and converts them to a zarr stored at https://dashboard.cloud.muni.cz/project/containers/container/Data
## Requires
* .aws folder with inside file named credentials with aws_access_key_id = and aws_secret_access_key =
* AWS CLI installed (for **aarch64** curl "https://awscli.amazonaws.com/awscli-exe-linux-aarch64.zip" -o "awscliv2.zip", unzip awscliv2.zip and ./aws/install)

In [3]:
pip install xarray h5netcdf s3fs tables zarr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 4.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numcodecs: filename=numcodecs-0.11.0-cp310-cp310-linux_aarch64.whl size=5275428 sha256=5306bb21202687279012cacde0bad2d1dbf700bf8a9dba65be198d7eeff4e213
  Stored in directory: /home/jeani/.cache/pip/wheels/c7/09/85/ea1c75772bdc41f5ea28af01031f10fa8faab856959b1ed148
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5050 sha256=c01c711c472d4637ac107740c561c32a4fcb67ce4037c1b8f4ff95c405b73265
  Stored in directory: /home/jeani/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built numcodecs asciitree
Note: you may 

In [4]:
import os
import s3fs
import xarray as xr
import zarr

In [5]:
local_path = '/home/jeani/data/'

In [6]:
# World Land cover data from 2015 to 2019 already available as a netCDF file stored locally
GLC_filename = os.path.join(local_path, 'C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc')
GLC_AOI = xr.open_dataset(GLC_filename)

In [7]:
%%time
GLC_AOI.load()

CPU times: user 30.4 s, sys: 8.45 s, total: 38.9 s
Wall time: 42.5 s


<xarray.Dataset>
Dimensions:                             (t: 5, x: 16709, y: 3064)
Coordinates:
  * t                                   (t) datetime64[ns] 2015-01-01 ... 201...
  * x                                   (x) float64 15.6 15.6 ... 31.06 31.07
  * y                                   (y) float64 71.19 71.19 ... 68.35 68.35
Data variables: (12/17)
    crs                                 |S1 b''
    Discrete_Classification_map         (t, y, x) float32 200.0 200.0 ... nan
    Discrete_Classification_proba       (t, y, x) float32 255.0 255.0 ... nan
    Forest_Type_layer                   (t, y, x) float32 255.0 255.0 ... nan
    Bare_CoverFraction_layer            (t, y, x) float32 255.0 255.0 ... nan
    Crops_CoverFraction_layer           (t, y, x) float32 255.0 255.0 ... nan
    ...                                  ...
    BuiltUp_CoverFraction_layer         (t, y, x) float32 255.0 255.0 ... nan
    PermanentWater_CoverFraction_layer  (t, y, x) float32 255.0 255.0 ... nan
    SeasonalWater_CoverFraction_layer   (t, y, x) float32 255.0 255.0 ... nan
    DataDensityIndicator                (t, y, x) float32 255.0 255.0 ... nan
    Change_Confidence_layer             (t, y, x) float32 255.0 255.0 ... nan
    dataMask                            (t, y, x) float32 nan nan ... nan nan
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform

In [8]:
# Hourly ERA5-land data from 2015-01-01 to 2022-12-31 already available as a netCDF file stored locally
ERA5_filename = os.path.join(local_path, 'reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.nc')
ERA5land = xr.open_dataset(ERA5_filename)

In [9]:
%%time
ERA5land.load()

CPU times: user 6.54 s, sys: 4.78 s, total: 11.3 s
Wall time: 12.7 s


<xarray.Dataset>
Dimensions:    (longitude: 155, latitude: 29, time: 70128, expver: 2)
Coordinates:
  * longitude  (longitude) float32 15.59 15.69 15.79 15.89 ... 30.79 30.89 30.99
  * latitude   (latitude) float32 71.15 71.05 70.95 70.85 ... 68.55 68.45 68.35
  * time       (time) datetime64[ns] 2015-01-01 ... 2022-12-31T23:00:00
  * expver     (expver) int32 1 5
Data variables:
    sd         (time, latitude, longitude) float32 nan nan ... 0.05434 0.05392
    t2m        (time, latitude, longitude, expver) float32 nan nan ... nan 269.2
    tp         (time, latitude, longitude, expver) float32 nan nan ... 0.0007116
Attributes:
    Conventions:  CF-1.6
    history:      2023-01-31 20:28:27 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

### Define s3 storage parameters

In [19]:
s3_path = 'Data'
s3_prefix =  "s3://" + s3_path
print(s3_prefix)
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}
print(access_key, secret_key)

s3://Data
e2d5c2e8bf9f4b3a857793176b723635 ad0478102b7142a2bd8248b6b98c13f7


### Write in remote S3 storage at https://object-store.cloud.muni.cz **as Zarr**

## Copernicus Global Land Cover data

In [20]:
GLC_zarr_name = GLC_filename.split('.nc')[0] + '.zarr'

In [21]:
url_zarr = os.path.join(s3_prefix, os.path.basename(GLC_zarr_name))
print(url_zarr)

s3://Data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.zarr


In [22]:
store_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
store_zarr.map.fs.find(url_zarr)

['Data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.zarr/.zgroup']

In [23]:
%time GLC_AOI.to_zarr(store=store_zarr, mode='w', consolidated=True)

CPU times: user 53.6 s, sys: 11.2 s, total: 1min 4s
Wall time: 13min 30s


## ERA5-land data

In [24]:
ERA5land_zarr_name = ERA5_filename.split('.nc')[0] + '.zarr'

In [25]:
url_zarr = os.path.join(s3_prefix, os.path.basename(ERA5land_zarr_name))
print(url_zarr)

s3://Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr


In [26]:
store_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
store_zarr.map.fs.find(url_zarr)

['Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/.zattrs',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/.zgroup',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/.zmetadata',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/expver/.zarray',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/expver/.zattrs',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/expver/0',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/latitude/.zarray',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/latitude/.zattrs',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/latitude/0',
 'Data/reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr/l

In [27]:
%time ERA5land.to_zarr(store=store_zarr, mode='w', consolidated=True)

CPU times: user 36.7 s, sys: 6.51 s, total: 43.2 s
Wall time: 6min 52s


In [28]:
print('Finished!')

Finished!
